# Reinforcement Learning assignment: Worm
### Made by Group 23:
- Kaj de Lange, 
- Troy Dijsselbloem, 
- Julian Dinnissen, s1135596
- Sten Nellen, 

In [9]:
import importlib
import numpy as np
import gymnasium as gym

import dp, mc, td
importlib.reload(dp)
importlib.reload(mc)
importlib.reload(td)
from dp import *
from mc import *
from td import *

# Abstract

_insert your abstract here_

# 0. Environment

In [2]:
from environment import CustomEnv
from gymnasium.envs.registration import register

    
register(id="worm-v0", entry_point="__main__:CustomEnv")


#Create the environment
env = gym.make("worm-v0")
# Interact with the environment
obs, info = env.reset()
for _ in range(10):
    action = env.action_space.sample() # Random action
    obs, reward, done, truncated, info = env.step(action)
    env.render()
    if done:
        break
env.close()

Current state: [5.5]
Current state: [5.]
Current state: [4.5]
Current state: [5.]
Current state: [5.5]
Current state: [6.]
Current state: [5.5]
Current state: [5.]
Current state: [4.5]
Current state: [4.]


c:\Users\User\Documents\GitHub\SOW-BKI258-Reinforcement-Learning\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:174: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed
  logger.warn(
c:\Users\User\Documents\GitHub\SOW-BKI258-Reinforcement-Learning\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be int8, actual type: float64
  logger.warn(
c:\Users\User\Documents\GitHub\SOW-BKI258-Reinforcement-Learning\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\User\Documents\GitHub\SOW-BKI258-Reinforcement-Learning\.venv\Lib\site-packages\gymnasium\utils\passive_env_check

# 1. Introduction

_insert your introduction in this cell_

_describe your environment and the problem the agent has to solve_

_describe the objective of the report (e.g. comparing various RL algorithms) and how you are going to accomplish this (research question)_

_don't forget to add plots/images of the environment, can be done via code cells, but also by inserting .png files into the jupyter notebook_

# 2. Dynamic Programming algorithms

First run the Dynamic Programming algorithms (Policy Iteration and Value Iteration) by calling functions from the separate `dp.py` file, and create plots. Then, fill in the cell completing your 'report' within this file.

In [3]:
# Call Policy Iteration algorithm

In [4]:
# Call Value Iteration algorithm

_In this cell, describe how the algorithms work, how the algorithms differ, plot results and/or policies (add more code cells!), etc._

# 3. Monte Carlo algorithms

First run the Monte Carlo algorithm (Monte Carlo Exploring Starts or Monte Carlo without Exploring Starts (with $\epsilon$-greedy strategy)) by calling functions from the separate `mc.py` file, and create plots. Then, fill in the cell completing your 'report' within this file.

In [5]:
# Call Monte Carlo algorithm

_In this cell, describe how the algorithms work, how the algorithms differ (compared to the one you did not code), plot results and/or policies (add more code cells!), etc._

# 4. Temporal Difference algorithms

First run the Temporal Difference algorithms (SARSA and Q-learning) by calling functions from the separate `td.py` file, and create plots. Then, fill in the cell completing your 'report' within this file.

In [6]:
# Call SARSA algorithm

In [7]:
# Call Q-learning algorithm

# 5. Comparison and discussion

Compare different algorithms (MC and TD with plots). You don’t need to plot DP alongside MC and TD since DP is not a learning algorithm. However, DP can provide the ground truth for optimal state or action values, which can serve as a reference when evaluating MC and TD. You can choose to plot any of the following: cumulative reward, root mean squared error, sample efficiency, or any other metric you think is a fair comparison.

Include a discussion: what can you conclude by comparing different RL algorithms? Do they have certain strengths or limitations?

_Add as many text cells as you like_

In [8]:
# Add as many code cells as you like

# 6. Conclusion

Conclude your project.

_Add as many text cells as you like_